<a href="https://colab.research.google.com/github/SelimSavas/AircraftSegmentationSAMandYOLOv8/blob/main/SAMandYOLOv8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# YOLOv8 vs SAM Meta

## Library

In [1]:
!pip install ultralytics
!pip install supervision
!pip install roboflow

import numpy as np
import cv2
import os
import sys
import torch
from roboflow import Roboflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.0/527.0 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 kB 9.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.2/56.2 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 8.8 MB/s eta 0:00:00
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9676 sha256=30efe2e953d6412157e36dc7594d8468d8329dbc9a069d7286395bfc4a144666
  Stored in directory: /root/.cache/p

In [2]:
HOME = os.getcwd()
print("HOME:", HOME)

HOME: /content


In [3]:
%cd {HOME}

!{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything.git'

/content
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-lxee1dqb
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-lxee1dqb
  Resolved https://github.com/facebookresearch/segment-anything.git to commit 567662b0fd33ca4b022d94d3b8de896628cd32dd
  Preparing metadata (setup.py) ... done
  Created wheel for segment-anything: filename=segment_anything-1.0-py3-none-any.whl size=36610 sha256=2c78c7fa487bf05e487a9b84b65c73ca7738197a85c1919fd531659ffabdee5d
  Stored in directory: /tmp/pip-ephem-wheel-cache-yyvnf9bh/wheels/d5/11/03/7aca746a2c0e09f279b10436ced7175926bc38f650b736a648
Successfully built segment-anything


In [4]:
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
MODEL_TYPE = "vit_h"

## Dataset (Roboflow)

In [5]:
rf = Roboflow(api_key="Pho4XVVaJnNcR3y3HNrK")
project = rf.workspace("team12").project("airplane-bp8fl")
dataset = project.version(2).download("yolov8")

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics<=8.0.20 is required but found version=8.0.81, to fix: `pip install ultralytics<=8.0.20`


Extracting Dataset Version Zip to airplane-2 in yolov8:: 100%|██████████| 297/297 [00:00<00:00, 2551.41it/s]


In [73]:
IMAGE_NAME = '/content/airplane3.jpg'
IMG_SOURCE = cv2.imread(IMAGE_NAME)

In [7]:
%cd {HOME}
!mkdir {HOME}/weights
%cd {HOME}/weights

!wget -q https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

/content
/content/weights


In [8]:
CHECKPOINT_PATH = os.path.join(HOME, "weights", "sam_vit_h_4b8939.pth")
print(CHECKPOINT_PATH, "; exist:", os.path.isfile(CHECKPOINT_PATH))

/content/weights/sam_vit_h_4b8939.pth ; exist: True


## YoloV8 - Creating BBox

In [74]:
from ultralytics import YOLO

yolo_model = YOLO('yolov8x.pt')
img = cv2.cvtColor(IMG_SOURCE, cv2.COLOR_BGR2RGB)
results = yolo_model.predict(source=img)


0: 416x640 25 persons, 2 airplanes, 81.9ms
Speed: 1.9ms preprocess, 81.9ms inference, 4.1ms postprocess per image at shape (1, 3, 640, 640)


In [75]:
boxes_class_name = np.array([])
for r in results:
    for c in r.boxes.cls:
        boxes_class_name = np.append(boxes_class_name,yolo_model.names[int(c)])
boxes_class_name

array(['airplane', 'airplane', 'person', 'person', 'person', 'person', 'person', 'person', 'person', 'person', 'person', 'person', 'person', 'person', 'person', 'person', 'person', 'person', 'person', 'person', 'person', 'person', 'person', 'person', 'person', 'person', 'person'], dtype='<U32')

In [77]:
k = 0
result_np_son = np.array([])
result_class = np.array([])
result_score = np.array([])
for result in results:
    result_np = np.array(result.boxes.data.cpu())

for i in result_np:
    result_np_son = np.append(result_np_son,i[:][:-2])
    result_class = np.append(result_class,i[:][-1:])
    result_score = np.append(result_score,i[:][-2:-1])
    k += 1


boxes = result_np_son.reshape((k,4))
boxes_class = result_class.reshape((k,))
boxes_score = result_score.reshape((k,))

In [78]:
boxes_class.shape

(27,)

In [79]:
type(boxes_class)

numpy.ndarray

In [80]:
boxes_class

array([          4,           4,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,
                 0,           0,           0])

In [81]:
boxes_class_value = {'0':sv.Color.white(),'2':sv.Color.blue(),'4':sv.Color.red(),'8':sv.Color.green()}

## SAM - Segmentation

In [82]:
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

sam = sam_model_registry[MODEL_TYPE](checkpoint=CHECKPOINT_PATH).to(device=DEVICE)

mask_predictor = SamPredictor(sam)

In [2]:
import supervision as sv

IMAGE_PATH = os.path.join(HOME, "data", IMAGE_NAME)
image_bgr = cv2.imread(IMAGE_PATH)
image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)

merged_mask = np.zeros(image_rgb.shape[:2], dtype=np.uint8)
segmented_image = image_bgr

#class_colors = (255, 0, 0)

i = 0

for box in boxes:
    mask_predictor.set_image(image_rgb)
    masks, scores, logits = mask_predictor.predict(
        box=box,
        multimask_output=True
    )
    print(box)
    print(scores)
    print(str(boxes_class[i]))
    print(boxes_class_name[i])

    #merged_mask = np.logical_or(merged_mask, masks)

    box_annotator = sv.BoxAnnotator(color= boxes_class_value[str(int(boxes_class[i]))], text_padding=20)
    mask_annotator = sv.MaskAnnotator(color= boxes_class_value[str(int(boxes_class[i]))])

    detections = sv.Detections(
        xyxy=sv.mask_to_xyxy(masks=masks),
        mask=masks
    )

    detections = detections[detections.area == np.max(detections.area)]

    source_image = box_annotator.annotate(scene=image_bgr.copy(), detections=detections,	 labels="d"+"d"+"10")
    i += 1
    segmented_image = mask_annotator.annotate(scene=segmented_image, detections=detections, opacity=0.7)

    sv.plot_images_grid(
      images=masks,
      grid_size=(1, 3),
      size=(8, 2)
    )

    sv.plot_images_grid(
        images=[image_bgr,source_image, segmented_image],
        grid_size=(1, 3),
        titles=['source image','source_image','segmented image'],
        size=(416,2)
    ) 

NameError: ignored